# HW1: Cassandra DB
ID1: 308564293

ID2: XXX

## Step 1: Setup Cassandra Cluster on Docker

We will install Cassandra on Docker using the code from the classroom. Following, we will open the container and interact with Cassandra.
The folllowing code should be executed only if there isn't a docker container with cassandra installed and running.

In [57]:
# On the first CMD:
# Pull the Cassandra Docker Image:
!docker pull cassandra:latest

# Start the first Cassandra node (Seed Node):
!docker run -d --name cass_cluster -p 9042:9042 cassandra:latest  

latest: Pulling from library/cassandra
Digest: sha256:5f20dbd2fc7f2061fc2a3c027e592193002f9eb475708bdb827bde615e7c1015
Status: Image is up to date for cassandra:latest
docker.io/library/cassandra:latest



What's Next?
  1. Sign in to your Docker account → docker login
  2. View a summary of image vulnerabilities and recommendations → docker scout quickview cassandra:latest


db484395c5b88f71364790e83519446c46a742482adff428305b828cd0808ac7


Install Depenencies, only if needed:

In [ ]:
# install cassandra driver
!pip3 install cassandra-driver

# install docker
!pip3 install docker

Import dependencies:

## Step 2: Data Ingestion

In [7]:
# Cassandra-driver
from cassandra.cluster import Cluster
from cassandra.query import SimpleStatement, BatchStatement

# Data
import csv
import pandas as pd

# Multithreading
import concurrent.futures

# Docker
import docker

Connect to the Cluster, Create keyspace and tables:

In [77]:
# Connect Cassandra-Driver to the Cluster running on the Docker:
cluster = Cluster(['127.0.0.1'])
session = cluster.connect()

From here, we can execute CQL commands from the notebook environment.

In [78]:
# Create Keyspace named 'books' and replication factor = 1
session.execute("CREATE KEYSPACE IF NOT EXISTS books WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 };")
session.execute("USE books")

# Validate the keyspace
keys_spaces = session.execute("DESCRIBE keyspaces;")
for ks in keys_spaces:
    print(ks)

Row(keyspace_name='books', type='keyspace', name='books')
Row(keyspace_name='system', type='keyspace', name='system')
Row(keyspace_name='system_auth', type='keyspace', name='system_auth')
Row(keyspace_name='system_distributed', type='keyspace', name='system_distributed')
Row(keyspace_name='system_schema', type='keyspace', name='system_schema')
Row(keyspace_name='system_traces', type='keyspace', name='system_traces')
Row(keyspace_name='system_views', type='keyspace', name='system_views')
Row(keyspace_name='system_virtual_schema', type='keyspace', name='system_virtual_schema')


We will set up a helper function to assist in executing an ingestion of a single row.

In [ ]:
# Insert 
def insert_row(row):
    """ Inserts a row from a CSV file to Cassandra
        Input:
        
    """
    session.execute(
        """
        INSERT INTO books (ISBN, Book_Title, Book_Author, Year_Of_Publication, Publisher, Image_URL_S, Image_URL_M, Image_URL_L)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """,
        (row[0], row[1], row[2], int(row[3]), row[4], row[5], row[6], row[7])
    )



Data ingestion: Books.CSV

In [ ]:
# Data Ingestion: books

# Create cassandra table with 8 columns
# ISBN is the partition key
session.execute("""
    CREATE TABLE IF NOT EXISTS books (
        ISBN text PRIMARY KEY,
        Book_Title text,
        Book_Author text,
        Year_Of_Publication int,
        Publisher text,
        Image_URL_S text,
        Image_URL_M text,
        Image_URL_L text
    )
""")

# Data ingestion for "books"
with open('Data/Books.csv', 'r') as f:
    next(f)  # Skip the header row.
    reader = csv.reader(f)

    with concurrent.futures.ThreadPoolExecutor() as executor:
        executor.map(insert_row, reader)

# Validation


In [ ]:
session.execute("""
    CREATE TABLE IF NOT EXISTS book_ratings (
        User_ID text,
        ISBN text,
        Book_Rating int,
        PRIMARY KEY(User_ID, ISBN)
    )
""")

with open('Data/Ratings.csv', 'r') as f:
    next(f)  # Skip the header row.
    reader = csv.reader(f)
    for row in reader:
        session.execute(
            """
            INSERT INTO book_ratings (User_ID, ISBN, Book_Rating)
            VALUES (%s, %s, %s)
            """,
            (row[0], row[1], int(row[2]))
        )

In [35]:
session.execute("""
    CREATE TABLE IF NOT EXISTS user_data (
        User_ID text PRIMARY KEY,
        Location text,
        Age float
    )
""")

def load_data(csv_file, insert_query):
        
    def insert_data(rows):
        batch = BatchStatement()
        for row in rows:
            age = float(row[2]) if row[2] else None
            batch.add(SimpleStatement(insert_query), (row[0], row[1], age))
        session.execute(batch)
        print("exec")
    
    with open(csv_file, "r") as f:
        next(f)  # Skip the header row.
        reader = list(csv.reader(f))
    
    # Adjust the number of rows for each batch and number of workers as needed
    num_rows_per_batch = 100
    rows = [reader[i: i + num_rows_per_batch] for i in range(0, len(reader), num_rows_per_batch)] 
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
        executor.map(insert_data, rows)



In [36]:
ratings_query =             """
            INSERT INTO book_ratings (User_ID, ISBN, Book_Rating)
            VALUES (%s, %s, %s)
            """
load_data('Data/Ratings.csv', ratings_query)

In [ ]:
users_query = """
                INSERT INTO user_data (User_ID, Location, Age)
                VALUES (%s, %s, %s)
                """
load_data('Data/Users.csv', users_query)

In [37]:
books_query = """
            INSERT INTO books (ISBN, Book_Title, Book_Author, Year_Of_Publication, Publisher, Image_URL_S, Image_URL_M, Image_URL_L)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
            """,
load_data('Data/Books.csv', books_query)

In [33]:
count_query = "SELECT COUNT(*) FROM user_data"
result = session.execute(count_query)
result.one()

Row(count=278858)

In [38]:
count_query = "SELECT COUNT(*) FROM books"
result = session.execute(count_query)
result.one()

Row(count=157963)

## Step 3: Creating Tables

In [ ]:
USE university;

In [ ]:
CREATE TABLE students (
  student_id UUID PRIMARY KEY,
  name TEXT,
  email TEXT,
  enrollment_year INT
);

In [ ]:
CREATE TABLE courses (
  course_id UUID PRIMARY KEY,
  course_name TEXT,
  lecturer_id UUID,
  credits INT
);

In [ ]:
CREATE TABLE lecturers (
  lecturer_id UUID PRIMARY KEY,
  name TEXT,
  department TEXT
);

In [ ]:
CREATE TABLE course_enrollments (
  course_id UUID,
  student_id UUID,
  enrollment_date DATE,
  PRIMARY KEY (course_id, student_id)
);

In [ ]:
# To validate you created the tables
cqlsh:university> DESCRIBE tables;

In [ ]:
# The output should be:
course_enrollments  courses  lecturers  students

## Step 4: Inserting data

In [ ]:
# Inserting data into students
INSERT INTO students (student_id, name, email, enrollment_year) VALUES (550e8400-e29b-41d4-a716-446655440000, 'John Doe', 'john.doe@email.com', 2021);
INSERT INTO students (student_id, name, email, enrollment_year) VALUES (652e8500-f39c-42d5-b517-557655450001, 'Jane Smith', 'jane.smith@email.com', 2020);
INSERT INTO students (student_id, name, enrollment_year) VALUES (753e8600-e49d-53e6-c628-668655460002, 'Alice Johnson', 2019);

In [ ]:
# Inserting data into lecturers
INSERT INTO lecturers (lecturer_id, name, department) VALUES (860e9700-a59e-63f7-d739-779655470003, 'Dr. James Watson', 'Biology');
INSERT INTO lecturers (lecturer_id, name, department) VALUES (961e9800-b69f-73c8-e840-889655480004, 'Dr. Emma Wilson', 'Physics');
INSERT INTO lecturers (lecturer_id, name) VALUES (072e9900-d710-83e9-f951-990655490005, 'Prof. Michael Brown');

In [ ]:
# Inserting data into courses
INSERT INTO courses (course_id, lecturer_id, course_name, credits) VALUES (183fa000-f821-93a0-b062-a96655650006, 860e9700-a59e-63f7-d739-779655470003, 'Biology 101', 3);
INSERT INTO courses (course_id, lecturer_id, course_name, credits) VALUES (284fb000-a932-a4b1-c173-b97755660007, 961e9800-b69f-73c8-e840-889655480004, 'Physics 101', 4);
INSERT INTO courses (course_id, lecturer_id, course_name, credits) VALUES (385fc000-d043-b5e2-f284-c98855670008, 072e9900-d710-83e9-f951-990655490005, 'Mathematics 101', 3);

In [ ]:
# Inserting data into course_enrollments
INSERT INTO course_enrollments (course_id, student_id, enrollment_date) VALUES (183fa000-f821-93a0-b062-a96655650006, 550e8400-e29b-41d4-a716-446655440000, '2023-10-01');
INSERT INTO course_enrollments (course_id, student_id, enrollment_date) VALUES (284fb000-a932-a4b1-c173-b97755660007, 652e8500-f39c-42d5-b517-557655450001, '2023-09-15');
INSERT INTO course_enrollments (course_id, student_id) VALUES (385fc000-d043-b5e2-f284-c98855670008, 753e8600-e49d-53e6-c628-668655460002);

## Step 5: Querying the data

In [1]:
# Get all students
SELECT * FROM STUDENTS

>>
 student_id                           | email                | enrollment_year | name
--------------------------------------+----------------------+-----------------+---------------
 753e8600-e49d-53e6-c628-668655460002 |                 null |            2019 | Alice Johnson
 652e8500-f39c-42d5-b517-557655450001 | jane.smith@email.com |            2020 |    Jane Smith
 550e8400-e29b-41d4-a716-446655440000 |   john.doe@email.com |            2021 |      John Doe
 (3 rows)

In [ ]:
# Get the list of students who enrolled after 2020 along with their email and the courses they enrolled in
SELECT student_id, name, email
FROM students
WHERE enrollment_year > 2020 ALLOW FILTERING;

>>
 student_id                           | name     | email
--------------------------------------+----------+--------------------
 550e8400-e29b-41d4-a716-446655440000 | John Doe | john.doe@email.com

(1 rows)

In [ ]:
# Find all courses and count of students enrolled in them
SELECT course_id, COUNT(student_id) as student_count
FROM course_enrollments
GROUP BY course_id;

>>
 course_id                            | student_count
--------------------------------------+---------------
 183fa000-f821-93a0-b062-a96655650006 |             1
 385fc000-d043-b5e2-f284-c98855670008 |             1
 284fb000-a932-a4b1-c173-b97755660007 |             1

(3 rows)